In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/ML Project'
!pwd

/content/drive/.shortcut-targets-by-id/11vZDVRncVAa-HSQzhEWDgaoMu2b1KfSR/ML Project
/content/drive/.shortcut-targets-by-id/11vZDVRncVAa-HSQzhEWDgaoMu2b1KfSR/ML Project


In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
import matplotlib.pyplot as plt


def plot_metric(hist, metric):
    plt.plot(hist.history[f"{metric}"], label=f"{metric}")
    plt.plot(hist.history[f"val_{metric}"], label=f"val_{metric}")
    plt.title(f"{metric} Function Evolution")
    plt.legend()


def plot_hist(hist):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plot_metric(hist, 'loss')

    plt.subplot(1, 2, 2)
    plot_metric(hist, 'accuracy')

In [ ]:
butterfly_path = 'butterfly-images40-species/'

In [ ]:
train_ds = image_dataset_from_directory(butterfly_path + 'train')


Found 9295 files belonging to 75 classes.


In [ ]:
validation_ds = image_dataset_from_directory(butterfly_path + 'valid')


Found 375 files belonging to 75 classes.


In [ ]:
test_ds = image_dataset_from_directory(
    butterfly_path + 'test',
    shuffle=False)  # when shuffle is True, it auto shuffles on each iteration


Found 375 files belonging to 75 classes.


In [ ]:
class_names = train_ds.class_names
class_count = len(class_names)
print(class_names)

['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO', 'ATALA', 'BANDED ORANGE HELICONIAN', 'BANDED PEACOCK', 'BECKERS WHITE', 'BLACK HAIRSTREAK', 'BLUE MORPHO', 'BLUE SPOTTED CROW', 'BROWN SIPROETA', 'CABBAGE WHITE', 'CAIRNS BIRDWING', 'CHECQUERED SKIPPER', 'CHESTNUT', 'CLEOPATRA', 'CLODIUS PARNASSIAN', 'CLOUDED SULPHUR', 'COMMON BANDED AWL', 'COMMON WOOD-NYMPH', 'COPPER TAIL', 'CRECENT', 'CRIMSON PATCH', 'DANAID EGGFLY', 'EASTERN COMA', 'EASTERN DAPPLE WHITE', 'EASTERN PINE ELFIN', 'ELBOWED PIERROT', 'GOLD BANDED', 'GREAT EGGFLY', 'GREAT JAY', 'GREEN CELLED CATTLEHEART', 'GREY HAIRSTREAK', 'INDRA SWALLOW', 'IPHICLUS SISTER', 'JULIA', 'LARGE MARBLE', 'MALACHITE', 'MANGROVE SKIPPER', 'MESTRA', 'METALMARK', 'MILBERTS TORTOISESHELL', 'MONARCH', 'MOURNING CLOAK', 'ORANGE OAKLEAF', 'ORANGE TIP', 'ORCHARD SWALLOW', 'PAINTED LADY', 'PAPER KITE', 'PEACOCK', 'PINE WHITE', 'PIPEVINE SWALLOW', 'POPINJAY', 'PURPLE HAIRSTREAK', 'PURPLISH COPPER', 'QUESTION MARK', 'RED ADMIRA

In [ ]:
# inputs must match the same form the model was originally trained on
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)
train_ds = train_ds.map(lambda image, label:
                        (tf.image.resize(image, size), label))
validation_ds = validation_ds.map(lambda image, label:
                                  (tf.image.resize(image, size), label))
test_ds = test_ds.map(lambda image, label:
                      (tf.image.resize(image, size), label))


In [ ]:
img_augmentation = tf.keras.models.Sequential(
    [
        tf.keras.layers.RandomRotation(factor=0.15),
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.RandomFlip(),
        tf.keras.layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
model = tf.keras.models.load_model('efficient_b0_30_epochs_scratch.h5')

## Try smaller learning rate

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate=.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
hist = model.fit(train_ds, validation_data=validation_ds, epochs=10)
plot_hist(hist)

Epoch 1/10
291/291 [==============================] - 2386s 8s/step - loss: 0.3994 - accuracy: 0.8936 - val_loss: 0.9747 - val_accuracy: 0.7947
Epoch 2/10
291/291 [==============================] - 2309s 8s/step - loss: 0.3710 - accuracy: 0.9012 - val_loss: 0.9612 - val_accuracy: 0.8080
Epoch 3/10
291/291 [==============================] - 2323s 8s/step - loss: 0.3348 - accuracy: 0.9125 - val_loss: 0.9430 - val_accuracy: 0.8187
Epoch 4/10
291/291 [==============================] - 2312s 8s/step - loss: 0.3259 - accuracy: 0.9159 - val_loss: 0.9368 - val_accuracy: 0.8213
Epoch 5/10
291/291 [==============================] - 2303s 8s/step - loss: 0.3078 - accuracy: 0.9187 - val_loss: 0.9134 - val_accuracy: 0.8240


KeyboardInterrupt: ignored

In [ ]:
model.save('efficient_b0_35_epochs_scratch.h5')

In [ ]:
y_pred = np.argmax(model.predict(test_ds), axis=-1)
y_test = np.concatenate(
    [label for batch, label in test_ds.as_numpy_iterator()])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      1.00      1.00         5
           2       1.00      0.60      0.75         5
           3       1.00      0.80      0.89         5
           4       0.71      1.00      0.83         5
           5       1.00      0.80      0.89         5
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         5
           8       0.33      0.40      0.36         5
           9       0.83      1.00      0.91         5
          10       0.67      0.80      0.73         5
          11       1.00      0.80      0.89         5
          12       1.00      0.80      0.89         5
          13       0.83      1.00      0.91         5
          14       0.80      0.80      0.80         5
          15       0.83      1.00      0.91         5
          16       1.00      0.60      0.75         5
          17       0.80    